### Creating connection to databases

In [1]:
from mysql_connection import MySqlManip
from neo4j_connection import Neo4jManip
import pandas as pd

mysql_conn = MySqlManip(
    user='root', host='10.60.5.99', port=3306, password='Boway@123', database='IOT'
)

neo4j_conn = Neo4jManip(
    uri = "bolt://10.60.5.99:7687",
    user = "neo4j",
    password = "Boway123"
)

# conn.create_params_table()

### Reading network structure from Neo4J

In [2]:
network_graph, sql_info = neo4j_conn.read_bayesian_graph("气泡失效")
print(f'''
network_graph: {network_graph},

"sql_info": {sql_info}      
      ''')


network_graph: [('镍', '气泡失效'), ('铝', '气泡失效'), ('镁', '气泡失效'), ('车间废料比例', '气泡失效'), ('锌', '气泡失效'), ('淘捡材料比例', '气泡失效'), ('外购材料比例', '气泡失效'), ('新金属材料比例', '气泡失效'), ('铣面屑料比例', '气泡失效'), ('硅', '气泡失效'), ('浇注温度_3米采样', '气泡失效'), ('浇注温度_开始采样', '气泡失效'), ('浇注温度_5米采样', '气泡失效'), ('总冷却强度', '气泡失效'), ('除气湿度', '气泡失效'), ('进水流量', '气泡失效'), ('湿度', '气泡失效')],

"sql_info": {'table_name': 'device_4_bubble_cleaned_data_bayes', '气泡失效': {'type': 'target', 'sql_column': 'label'}, '镍': {'type': 'factor', 'sql_column': 'NI', 'upper': 2.35, 'lower': 2.25}, '铝': {'type': 'factor', 'sql_column': 'AL', 'upper': 0.002, 'lower': 0}, '镁': {'type': 'factor', 'sql_column': 'MG', 'upper': 0.15, 'lower': 0.08}, '车间废料比例': {'type': 'factor', 'sql_column': 'weight_percent_waste', 'upper': 100, 'lower': 0}, '锌': {'type': 'factor', 'sql_column': 'ZN', 'upper': 0.07, 'lower': 0}, '淘捡材料比例': {'type': 'factor', 'sql_column': 'weight_percent_pick', 'upper': 100, 'lower': 0}, '外购材料比例': {'type': 'factor', 'sql_column': 'weight_percent_outside_

### Reading structured data from MySQL

In [3]:
raw_data = mysql_conn.load_data_to_dataframe(sql_info["table_name"])
raw_data = raw_data.dropna()
series = []

for variable in sql_info.keys():
    if variable == "table_name":
        continue
    ser = raw_data[sql_info[variable]["sql_column"]]
    ser.name = variable
    if sql_info[variable]["type"] == "target":
        ser = pd.cut(
            ser, 
            2,
            labels=["FineBatch", "BadBatch"])
    else:
        try:
            ser = pd.cut(
                ser.astype('float'), 
                [-float("inf"), 
                float(sql_info[variable]["lower"]), 
                float(sql_info[variable]["upper"]),
                float("inf")],
                labels=["lower", "medium", "upper"])
        except Exception as e:
            print(f"{variable}: {e}")
    series.append(ser)

data = pd.DataFrame(series).T
data


,气泡失效,镍,铝,镁,车间废料比例,锌,淘捡材料比例,外购材料比例,新金属材料比例,铣面屑料比例,硅,浇注温度_3米采样,浇注温度_开始采样,浇注温度_5米采样,总冷却强度,除气湿度,进水流量,湿度
0,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,lower,upper
1,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,lower,upper
2,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,lower,upper
3,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,lower,upper
4,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,lower,upper
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16208,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,medium,upper
16209,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,medium,upper
16210,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,medium,upper
16211,FineBatch,medium,upper,medium,medium,medium,lower,lower,medium,medium,medium,medium,medium,medium,lower,lower,medium,upper


### Creating model

In [4]:
from models.bayes_net import BNCreator
model = BNCreator(network_graph)
# model.to_graphviz()

c:\Users\pw\Desktop\ml_kg_connection\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


TypeError: 'ellipsis' object is not iterable

training model

In [5]:
# from pgmpy.estimators import ExpectationMaximization

# estimator = ExpectationMaximization(model, data)
# latent_card = {}
# for latent_node in latents:
#     latent_card[latent_node] = 10
# params = estimator.get_parameters(latent_card = latent_card)
# model.add_cpds(*params)

model.fit(data)

### Making inference

In [14]:
sample_data = data.drop(columns=["气泡失效"]).loc[[499]]

model.predict_probability(sample_data)


,气泡失效_BadBatch,气泡失效_FineBatch
499,0.141246,0.858754


### Input interface

In [17]:
model.make_inference({"气泡失效": 1, "镁": 10.5, "车间废料比例": 100})

,气泡失效
1,1003
